In [15]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import json
import xmltodict
import pandas as pd

In [16]:
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
serviceKey = '3f%2FnmTs7kFeKIVO%2FQYey91YeNYJL5W7HtOKfhVj3cr4a7coxcZmXf6zHssAg2pa9gMjgTxE%2FfpT%2B8CfWb2NEWA%3D%3D'
pageNo = '1'
numOfRows = '10'
_type = 'xml'

In [17]:
# 해당 컬럼 작성
coloumns = ["resultCode"
           , "resultMsg"
           , "numOfRows"
           , "pageNo"
           , "totalCount"
           , "seq"
           , "stateDt"
           , "stateTime"
           , "decideCnt"
           , "clearCnt"
           , "examCnt"
           , "deathCnt"
           , "careCnt"
           , "resutlNegCnt"
           , "accExamCnt"
           , "accExamCompCnt"
           , "accDefRate"
           , "createDt"
           , "updateDt"]

In [18]:
# 
def return_list_from_respose(List) -> list :
    return [
         List["response"]["header"]["resultCode"]
        ,List["response"]["header"]["resultMsg"]
        ,List["response"]["body"]["item"]["numOfRows"]
        ,List["response"]["body"]["items"]["pageNo"]
        ,List["response"]["body"]["items"]["totalCount"]
        ,List["response"]["body"]["items"]["item"]["seq"]
        ,List["response"]["body"]["items"]["item"]["stateDt"]
        ,List["response"]["body"]["items"]["item"]["stateTime"]
        ,List["response"]["body"]["items"]["item"]["decideCnt"]
        ,List["response"]["body"]["items"]["item"]["clearCnt"]
        ,List["response"]["body"]["items"]["item"]["examCnt"]
        ,List["response"]["body"]["items"]["item"]["deathCnt"]
        ,List["response"]["body"]["items"]["item"]["careCnt"]
        ,List["response"]["body"]["items"]["item"]["resutlNegCnt"]
        ,List["response"]["body"]["items"]["item"]["accExamCnt"]
        ,List["response"]["body"]["items"]["item"]["accExamCompCnt"]
        ,List["response"]["body"]["items"]["item"]["accDefRate"]
        ,List["response"]["body"]["items"]["item"]["createDt"]
        ,List["response"]["body"]["items"]["item"]["updateDt"]
    ]

In [19]:
# parse 합치기
def concat_parse(url, serviceKey, pageNo, numOfRows, _type) -> str :
    return url + \
     "?serviceKey=" + serviceKey + \
     "&pageNo=" + pageNo + \
     "&numOfRows=" + numOfRows + \
     "&type=" + _type

In [20]:
# parse check
tot = concat_parse(url, serviceKey, pageNo, numOfRows, _type)
print(tot)

http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=3f%2FnmTs7kFeKIVO%2FQYey91YeNYJL5W7HtOKfhVj3cr4a7coxcZmXf6zHssAg2pa9gMjgTxE%2FfpT%2B8CfWb2NEWA%3D%3D&pageNo=1&numOfRows=10&type=xml


In [21]:
error_code = [0, 1, 2, 3, 4
             , 5, 10, 11, 12, 20, 21, 22
             , 30, 31, 32, 33, 99]
res = []
idx = 1

try :        
    while idx < 2 :        
        print("idx : ", idx)
        
        pageNo = str(idx)
        tot = concat_parse(url, serviceKey, pageNo, numOfRows, _type)
        response = urlopen(tot)
#         response = requests.get(url).json()
        
        getCode = response.getcode()
        if getCode in error_code :
            print("getcode() : ", getCode, " => break!")
            break
            
        
        response_body = response.read().decode('utf-8')
        jsonString = json.dumps(xmltodict.parse(response_body))        
        json_object = json.loads(jsonString)
        print(json_object)
        tmp = return_list_from_respose(json_object)
        print(tmp)
        res.extend(tmp)    
        
        print(idx," 완료")
        idx += 1
        print("================")
        
except Exception as ex:
    # 여기서 진짜 에러와 끝이란 걸 어떻게 알 수 있지?
    # 일단, 끝이 있으니까 그만 돌게 한 의도는 설계 완료!
    print("error : ", ex)

idx :  1
{'response': {'header': {'resultCode': '99', 'resultMsg': 'LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.'}}}
error :  'body'


In [ ]:
# 테이블 변환
parsing_table = pd.DataFrame(res, columns = coloumns)
parsing_table.head()

In [ ]:
parsing_table.info()